In [45]:
using MeshCatMechanisms
using MeshCat
using LinearAlgebra
using RigidBodyDynamics
using Plots
using GeometryTypes
using CoordinateTransformations
using TrajectoryOptimization

Load Kuka iiwa model (from URDF)

In [46]:
model = Dynamics.kuka_model
n,m = model.n, model.m

(14, 7)

Numerical type

T = Float64

In [5]:
function hold_trajectory(n,m,N, mech::Mechanism, q)
    state = MechanismState(mech)
    nn = num_positions(state)
    set_configuration!(state, q[1:nn])
    vd = zero(state.q)
    u0 = dynamics_bias(state)

    
    if length(q) > m
        throw(ArgumentError("system must be fully actuated to hold an arbitrary position ($(length(q)) should be > $m)"))
    end
    U0 = zeros(m,N)
    for k = 1:N
        U0[:,k] = u0
    end
    return U0
end


hold_trajectory (generic function with 1 method)

Initial and final states

In [47]:
x0 = zeros(n)

xf = zeros(n)
xf[1] = pi/2
xf[2] = pi/2;

Cost

In [39]:
Q = Diagonal([ones(7); ones(7)*100])
Qf = 1000.0*Diagonal(I,n)
R = 1e-2*Diagonal(I,m)
tf = 5.0
_cost = LQRCost(Q, R, Qf, xf)
xf_ee = ee_fun(xf)
x0_ee = ee_fun(x0);

Solver options

In [48]:
verbose=false
opts_ilqr = iLQRSolverOptions{T}(verbose=true,iterations=300,live_plotting=:off);
opts_al = AugmentedLagrangianSolverOptions{T}(verbose=true,opts_uncon=opts_ilqr,
    iterations=20,cost_tolerance=1.0e-6,cost_tolerance_intermediate=1.0e-5,constraint_tolerance=1.0e-3,penalty_scaling=50.,penalty_initial=0.01)
opts_altro = ALTROSolverOptions{T}(verbose=true,resolve_feasible_problem=false,opts_al=opts_al,R_inf=0.01);

Terminal constraint

In [41]:
goal = goal_constraint(xf);

Problem

In [49]:
N = 51 # number of knot points
dt = tf/(N-1)# total time

U_hold = hold_trajectory(n,m,N, kuka, x0[1:7])
obj = Objective(_cost,N) # objective with same stagewise costs
con = ProblemConstraints([goal],N)

prob = Problem(model,obj, x0=x0, integration=:rk4, N=N, dt=dt)
initial_controls!(prob, U_hold); # initialize problem with controls

In [43]:
solve!(prob, opts_al)

    iter cost          expected    z         α         ρ         dJ        grad      zero_count info                                              
____----------------------------------------------------------------------------------------------------------------------------------------------
    1     1443.1376597  1068.09329  0.961275  0.25      0e+00     1026.731  4.239299 0          
    2     846.93520945  619.241079  0.962795  0.25      0e+00     596.2025  4.18128  0          
    3     299.09175532  615.343795  0.890305  0.5       0e+00     547.8435  4.312714 0          
    4     109.70201339  205.823184  0.920157  0.5       0e+00     189.3897  4.516447 0          
    5     36.251166991  85.7232101  0.856837  1         0e+00     73.45085  2.754992 0          
    6     26.365147698  12.5997681  0.784619  1         0e+00     9.886019  0.074719 0          
    7     23.763815218  2.72398199  0.954974  1         0e+00     2.601332  0.029682 0          
    8     23.640448397  0.1

23.64033373346675

Visualizer

In [53]:
kuka = parse_urdf(Dynamics.urdf_kuka,remove_fixed_tree_joints=false)
kuka_visuals = URDFVisuals(Dynamics.urdf_kuka)
state = MechanismState(kuka)
world = root_frame(kuka)

# Create Visualizer
vis = Visualizer()
mvis = MechanismVisualizer(kuka, kuka_visuals, vis[:base])
# open(vis)
IJuliaCell(vis)

WebIO.Scope("scope-ca0df22d-d9c8-4d4a-b9ec-82aa42f8a938", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :iframe), Any[], Dict{Symbol,Any}(:attributes=>Dict{String,Any}("height"=>"100%","scrolling"=>"no","src"=>"javascript:void(0)","frameborder"=>0),:id=>"ifr",:style=>Dict("width"=>"100%","flexGrow"=>"1")), 0)], Dict{Symbol,Any}(:style=>Dict("flexDirection"=>"column","height"=>"100%","minHeight"=>"400px","display"=>"flex","width"=>"100%","overflow"=>"hidden")), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}(), Set(String[]), nothing, Any[], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function () {\n    var frame = this.dom.querySelector(\"#ifr\");\n    var doc = frame.contentDocument\n    var win = frame.contentWindow\n\n    // Ensure that the iframe's baseURI matches the baseURI of the\n    // outer document. This is necessary to resolve\n    // https://github.com/JuliaGizmos/WebIO.jl/issues/167\n    var base = doc.createElement(\"base\");\n    base.setAttribute(\"href\", document.baseURI);\n    doc.head.appendChild(base);\n\n    var webio = doc.createElement(\"script\")\n    webio.src = \"/assetserver/c443b012d0ad3ce8e6cc9de306cd3766b1d18a79-assets/webio/dist/bundle.js\"\n    var parent = window\n\n    function resizeIframe() {\n        doc.body.style.padding = '0'\n        doc.body.style.margin = '0'\n        doc.documentElement.height = '100%'\n        doc.body.height = '100%'\n    }\n\n    webio.onload = function () {\n        win.WebIO.sendCallback = parent.WebIO.sendCallback; // Share stuff\n        win.WebIO.scopes = parent.WebIO.scopes\n        win.WebIO.obsscopes = parent.WebIO.obsscopes\n        win.WebIO._connected = true\n        doc.body.innerHTML = \"<html><body>\" + \"<div class='display:none'></div><unsafe-script style='display:none'>\\nWebIO.mount(this.previousSibling,{&quot;props&quot;:{},&quot;nodeType&quot;:&quot;Scope&quot;,&quot;type&quot;:&quot;node&quot;,&quot;instanceArgs&quot;:{&quot;imports&quot;:{&quot;data&quot;:[{&quot;name&quot;:null,&quot;type&quot;:&quot;js&quot;,&quot;url&quot;:&quot;/assetserver/39091d6b9eb892b592bedf9cea6c288f61f7d3f3-main.min.js&quot;}],&quot;type&quot;:&quot;async_block&quot;},&quot;id&quot;:&quot;scope-ecf28558-e306-46fd-a9a1-22e35ac4468f&quot;,&quot;handlers&quot;:{&quot;meshcat-command&quot;:[(function (val){return this.viewer.handle_command_message({&quot;data&quot;:val})})],&quot;_promises&quot;:{&quot;importsLoaded&quot;:[(function (mc){var element=this.dom.children[0]; this.viewer=new mc.Viewer(element); WebIO.setval({&quot;name&quot;:&quot;meshcat-request&quot;,&quot;scope&quot;:&quot;scope-ecf28558-e306-46fd-a9a1-22e35ac4468f&quot;,&quot;id&quot;:&quot;ob_11&quot;,&quot;type&quot;:&quot;observable&quot;},String(Date.now())); window.document.body.style.margin=&quot;0&quot;; return window.meshcat_viewer=this.viewer})]}},&quot;systemjs_options&quot;:null,&quot;observables&quot;:{&quot;meshcat-command&quot;:{&quot;sync&quot;:false,&quot;id&quot;:&quot;ob_10&quot;,&quot;value&quot;:[131,166,109,97,116,114,105,120,199,64,23,141,72,172,44,126,73,238,146,0,0,128,63,0,0,0,0,11,5,191,128,0,0,128,63,126,73,238,146,0,0,0,0,0,0,128,191,172,75,19,3,141,72,172,44,0,0,0,0,0,0,0,16,0,0,0,146,236,81,56,61,0,0,128,63,164,116,121,112,101,173,115,101,116,95,116,114,97,110,115,102,111,114,109,164,112,97,116,104,218,0,133,47,109,101,115,104,99,97,116,47,98,97,115,101,47,119,111,114,108,100,47,98,97,115,101,47,105,105,119,97,95,108,105,110,107,95,48,47,105,105,119,97,95,108,105,110,107,95,49,47,105,105,119,97,95,108,105,110,107,95,50,47,105,105,119,97,95,108,105,110,107,95,51,47,105,105,119,97,95,108,105,110,107,95,52,47,105,105,119,97,95,108,105,110,107,95,53,47,105,105,119,97,95,108,105,110,107,95,54,47,105,105,119,97,95,108,105,110,107,95,55,47,105,105,119,97,95,108,105,110,107,95,101,101]},&quot;meshcat-controls&quot;:{&quot;sync&quot;:true,&quot;id&quot;:&quot;ob_12&quot;,&quot;value&quot;:[]},&quot;me

Create animation

In [54]:
q = [prob.X[k][1:7] for k = 1:N]
t = range(0,stop=tf,length=N)

setanimation!(mvis,t,q)